In [17]:
import cv2
import random
import time
from keras.models import load_model
import numpy as np
model = load_model("model.savedmodel")
cap = cv2. VideoCapture(0)
data = np.ndarray(shape=(1, 224, 224, 3), dtype = np.float32)

player_score = 0
computer_score = 0

def user(prediction, player_win, computer_win):
    
        if prediction[0,0] > 0.5:
            user_input = "rock"
        elif prediction[0,1] > 0.5:
            user_input = "paper"
        elif prediction[0,2] > 0.5:
            user_input = "scissors"
        else:
            user_input = "nothing"

        comp_option = ["rock", "paper", "scissors"]
        comp_input= random.choice(comp_option)

        if user_input == "rock" and comp_input == "rock":
            message = 'You chose rock. Computer chose rock. Draw!'
        elif user_input == "rock" and comp_input == "paper":
            message = 'You chose rock. Computer chose paper. You lost!'
            computer_win = True
        elif user_input == "rock" and comp_input == "scissors":
            message = 'You chose rock. Computer chose scissors. You won!'
            player_win += True
        elif user_input == "paper" and comp_input == "rock":
            message = 'You chose paper. Computer chose rock. You won!'
            player_win += True
        elif user_input == "paper" and comp_input == "paper":
            message = 'You chose paper. Computer chose paper. Draw!'
        elif user_input == "paper" and comp_input == "scissors":
            message = 'You chose paper. Computer chose scissors. You lost!'
            computer_win += True
        elif user_input == "scissors" and comp_input == "rock":
            message = 'You chose scissors. Computer chose rock. You lost!'
            computer_win += True
        elif user_input == "scissors" and comp_input == "paper":
            message = 'You chose scissors. Computer chose paper. You won!'
            player_win += True
        elif user_input == "scissors" and comp_input == "scissors":
            message = 'You chose scissors. Computer chose scissors. Draw!'
        else:
            message = "You didn't pick anything!"

        return message


cam_started = False
game_started = False
countdown = False
counter = 0
elapsed = 0
game_over = False
player_win = False
computer_win = False



while True:
    ret, frame = cap.read()
    if not cam_started:
        message = "Press 'a' to start! Or 'q' to quit."
    if cv2.waitKey(33) == ord('a'):
        if not cam_started:
            counter = time.time()
            started = True
            countdown = True
            message = "Rock, Paper or Scissors?"
            cam_started = True
    font = cv2.FONT_HERSHEY_TRIPLEX
    cv2.putText(frame, message, (50,50), font, 0.6, (255,0,255), 2, cv2.LINE_4)
    resized_frame = cv2.resize(frame,(224,224),interpolation = cv2. INTER_AREA)
    image_np = np.array (resized_frame)
    normalized_image = (image_np.astype(np.float32)/127.0) - 1
# Normalize the image
    data[0] = normalized_image
    prediction = model.predict(data)
    cv2.imshow("frame", frame)

    elapsed = time.time() - counter

    if elapsed >= 3 and elapsed < 1646691903 and game_started == False and game_over == False:
        game_started = True
        elapsed = time.time()-counter
        user(prediction, player_win, computer_win)
    # press q to close the window
        message = user(prediction, player_win, computer_win)

        if message == 'You chose rock. Computer chose scissors. You won!' or message == 'You chose paper. Computer chose rock. You won!' or message == 'You chose scissors. Computer chose paper. You won!':
            player_score +=1
        elif message == "You didn't pick anything!":
            pass
        elif message == 'You chose rock. Computer chose rock. Draw!' or message == 'You chose paper. Computer chose paper. Draw!' or message == 'You chose scissors. Computer chose scissors. Draw!' :
            pass
        else:
            computer_score +=1

    if elapsed >= 6 and elapsed < 1646691903 and game_started == True and game_over == False:
        message = f"You: {player_score} Computer: {computer_score}"

    if elapsed >= 9 and elapsed < 1646691903 and game_started == True and game_over == False:
        message = "Press 'n' for the next round"


    if cv2.waitKey(1) & 0xFF == ord("n") and game_started == True and game_over == False:
       if cam_started:
            counter = time.time()
            game_started = False
            message = "Rock, Paper or Scissors?"
    
    

    if player_score == 3:
        message = "You won 3 rounds first! Well Done!"
        game_over = True
    elif computer_score == 3:
        message = "Computer won 3 rounds first. Game over."
        game_over = True
   
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# after the loop release the cap object
cap.release()
# destroy all windows
cv2.destroyAllWindows() 






